In [ ]:
"""
=============================================================================
BIOMEDICAL RAG SYSTEM - OPTIMIZED VERSION
Neo4j Knowledge Graph + Multi-LLM Backend
Improvements: Enhanced search, better context retrieval, relevance ranking
=============================================================================
"""

# =============================================================================
# CELL 1: INSTALL DEPENDENCIES
# =============================================================================

!pip install neo4j requests -q

# =============================================================================
# CELL 2: IMPORT LIBRARIES
# =============================================================================

import requests
import json
from typing import Optional, Dict, Any, List
from neo4j import GraphDatabase
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully")

# =============================================================================
# CELL 3: ENHANCED NEO4J CONNECTOR CLASS
# =============================================================================

class Neo4jConnector:
    """Connect to Neo4j Aura and retrieve biomedical context with improved search."""

    def __init__(self, uri: str, username: str, password: str, database: str = "neo4j"):
        self.driver = GraphDatabase.driver(uri, auth=(username, password))
        self.database = database
        print(f"✓ Connected to Neo4j at {uri}")

    def close(self):
        self.driver.close()
        print("✓ Neo4j connection closed")

    def query(self, cypher: str, parameters: Optional[Dict] = None) -> List[Dict]:
        """Execute Cypher query and return results."""
        with self.driver.session(database=self.database) as session:
            result = session.run(cypher, parameters or {})
            return [record.data() for record in result]

    def get_schema(self) -> str:
        """Retrieve database schema."""
        try:
            labels_query = "CALL db.labels() YIELD label RETURN collect(label) as labels"
            rels_query = "CALL db.relationshipTypes() YIELD relationshipType RETURN collect(relationshipType) as types"

            labels = self.query(labels_query)
            rels = self.query(rels_query)

            schema = {
                "node_labels": labels[0]['labels'] if labels else [],
                "relationship_types": rels[0]['types'] if rels else []
            }
            return json.dumps(schema, indent=2)
        except Exception as e:
            return f"Schema retrieval error: {e}"

    def get_sample_data(self, limit: int = 5) -> str:
        """Get sample nodes to understand data structure."""
        query = f"""
        MATCH (n)
        RETURN labels(n) AS labels, properties(n) AS props
        LIMIT {limit}
        """
        results = self.query(query)
        return json.dumps(results, indent=2)

    def search_entity_enhanced(self, entity_name: str, limit: int = 15) -> str:
        """Enhanced search with relevance scoring and multi-hop traversal."""
        query = """
        MATCH (n)
        WHERE (n.text IS NOT NULL AND toLower(n.text) CONTAINS toLower($entity))
           OR (n.name IS NOT NULL AND toLower(n.name) CONTAINS toLower($entity))
           OR (n.canonical_name IS NOT NULL AND toLower(n.canonical_name) CONTAINS toLower($entity))
        WITH n,
             CASE
               WHEN toLower(n.name) = toLower($entity) THEN 10
               WHEN toLower(n.canonical_name) = toLower($entity) THEN 10
               WHEN toLower(n.name) CONTAINS toLower($entity) THEN 5
               WHEN toLower(n.canonical_name) CONTAINS toLower($entity) THEN 5
               WHEN toLower(n.text) CONTAINS toLower($entity) THEN 1
               ELSE 0
             END AS relevance_score
        ORDER BY relevance_score DESC
        LIMIT $limit
        OPTIONAL MATCH (n)-[r]->(m)
        RETURN n, labels(n) AS node_labels, r, type(r) AS rel_type, m, labels(m) AS target_labels, relevance_score
        """

        results = self.query(query, {"entity": entity_name, "limit": limit})

        if not results:
            return f"No results found for '{entity_name}'."

        context_parts = []
        seen_nodes = set()

        for idx, record in enumerate(results, 1):
            node = record.get('n', {})
            node_id = id(node)

            if node_id in seen_nodes:
                continue
            seen_nodes.add(node_id)

            node_labels = record.get('node_labels', [])
            rel_type = record.get('rel_type')
            target = record.get('m')
            target_labels = record.get('target_labels', [])
            relevance = record.get('relevance_score', 0)

            node_props = {k: v for k, v in dict(node).items()
                         if k != 'embedding' and isinstance(v, (str, int, float, bool))}

            label = node_labels[0] if node_labels else "Node"
            text_prop = node_props.get('text') or node_props.get('name') or node_props.get('canonical_name') or ''

            if text_prop and len(text_prop) > 300:
                text_prop = text_prop[:300] + "..."

            if text_prop:
                relevance_indicator = "★" * min(3, int(relevance / 3)) if relevance > 0 else ""
                context_parts.append(f"{idx}. [{label}] {relevance_indicator} {text_prop}")

            if rel_type and target:
                target_props = {k: v for k, v in dict(target).items()
                               if k != 'embedding' and isinstance(v, (str, int, float, bool))}
                target_text = target_props.get('text') or target_props.get('name') or target_props.get('canonical_name') or ''

                if target_text and len(target_text) > 200:
                    target_text = target_text[:200] + "..."

                if target_text:
                    target_label = target_labels[0] if target_labels else "Node"
                    context_parts.append(f"   → {rel_type}: [{target_label}] {target_text}")

        return "\n".join(context_parts)

    def search_entity_multihop(self, entity_name: str, max_hops: int = 2, limit: int = 20) -> str:
        """Multi-hop search to get richer context."""
        query = f"""
        MATCH path = (n)-[*1..{max_hops}]->(m)
        WHERE (n.text IS NOT NULL AND toLower(n.text) CONTAINS toLower($entity))
           OR (n.name IS NOT NULL AND toLower(n.name) CONTAINS toLower($entity))
           OR (n.canonical_name IS NOT NULL AND toLower(n.canonical_name) CONTAINS toLower($entity))
        WITH n, path, m, length(path) as path_length
        ORDER BY path_length ASC
        LIMIT $limit
        RETURN n, labels(n) as n_labels, relationships(path) as rels, m, labels(m) as m_labels
        """

        results = self.query(query, {"entity": entity_name, "limit": limit})

        if not results:
            return f"No multi-hop results found for '{entity_name}'."

        context_parts = []

        for idx, record in enumerate(results[:10], 1):
            node = record.get('n', {})
            target = record.get('m', {})
            rels = record.get('rels', [])
            n_labels = record.get('n_labels', [])
            m_labels = record.get('m_labels', [])

            node_props = {k: v for k, v in dict(node).items()
                         if k != 'embedding' and isinstance(v, (str, int, float, bool))}
            target_props = {k: v for k, v in dict(target).items()
                           if k != 'embedding' and isinstance(v, (str, int, float, bool))}

            n_label = n_labels[0] if n_labels else "Node"
            m_label = m_labels[0] if m_labels else "Node"

            n_text = (node_props.get('text') or node_props.get('name') or node_props.get('canonical_name') or '')[:150]
            m_text = (target_props.get('text') or target_props.get('name') or target_props.get('canonical_name') or '')[:150]

            if n_text and m_text:
                rel_chain = " → ".join([type(r).__name__ for r in rels]) if rels else "connected to"
                context_parts.append(f"{idx}. [{n_label}] {n_text} --{rel_chain}--> [{m_label}] {m_text}")

        return "\n".join(context_parts)

    def search_entity(self, entity_name: str, limit: int = 15, use_multihop: bool = False) -> str:
        """Unified search method with optional multi-hop."""
        if use_multihop:
            return self.search_entity_multihop(entity_name, max_hops=2, limit=limit)
        else:
            return self.search_entity_enhanced(entity_name, limit=limit)

    def custom_query_to_context(self, cypher: str, parameters: Optional[Dict] = None) -> str:
        """Execute custom Cypher and format as context."""
        results = self.query(cypher, parameters)

        if not results:
            return "No results returned from query."

        context_parts = []
        for i, record in enumerate(results[:15], 1):
            clean_record = {}
            for k, v in record.items():
                if isinstance(v, dict):
                    clean_v = {kk: vv for kk, vv in v.items()
                              if kk != 'embedding' and isinstance(vv, (str, int, float, bool))}
                    clean_record[k] = clean_v
                else:
                    clean_record[k] = v
            context_parts.append(f"{i}. {json.dumps(clean_record, indent=2)}")

        return "\n".join(context_parts)

print("✓ Enhanced Neo4jConnector class defined")

# =============================================================================
# CELL 4: OPTIMIZED RAG LLM BACKEND CLASS
# =============================================================================

class BiomedicalRAG:
    """RAG system with improved prompting and model management."""

    def __init__(self, backend: str = "groq", api_key: Optional[str] = None, model: Optional[str] = None):
        self.backend = backend
        self.api_key = api_key

        self.configs = {
            "groq": {
                "url": "https://api.groq.com/openai/v1/chat/completions",
                "headers": {
                    "Authorization": f"Bearer {api_key}",
                    "Content-Type": "application/json"
                } if api_key else {},
                "model": model or "llama-3.1-8b-instant",
                "available_models": [
                    "llama-3.1-8b-instant",
                    "llama-3.1-70b-versatile",
                    "llama-3.2-1b-preview",
                    "llama-3.2-3b-preview",
                    "llama-3.2-11b-vision-preview",
                    "llama-3.2-90b-vision-preview",
                    "gemma2-9b-it",
                    "mixtral-8x7b-32768"
                ]
            },
            "huggingface": {
                "url": "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2",
                "headers": {"Authorization": f"Bearer {api_key}"} if api_key else {}
            },
            "ollama": {
                "url": "http://localhost:11434/api/generate",
                "model": model or "llama3.1"
            }
        }

        current_model = self.configs[backend].get("model", "default")
        print(f"✓ RAG system initialized with {backend} backend")
        print(f"  Model: {current_model}")

    def _build_prompt(self, question: str, context: str, max_context_length: int = 4500) -> str:
        """Build enhanced prompt with better instructions."""
        if len(context) > max_context_length:
            context = context[:max_context_length] + "\n... (context truncated)"

        prompt = f"""You are an expert biomedical AI assistant with deep knowledge of medicine, pharmacology, and biology.

Your task is to answer the question using ONLY the information provided in the Knowledge Graph context below.

KNOWLEDGE GRAPH CONTEXT:
{context}

QUESTION: {question}

INSTRUCTIONS:
- Synthesize information from multiple context entries if relevant
- Provide specific, detailed answers when context is available
- Explain mechanisms, relationships, and connections clearly
- If context is insufficient, state what information is missing
- Be precise and use medical terminology appropriately
- Cite which context entries you used (e.g., "Based on entry 1 and 3...")

ANSWER:"""

        return prompt

    def _query_groq(self, prompt: str) -> str:
        """Query Groq API with enhanced error handling."""
        config = self.configs["groq"]

        payload = {
            "model": config["model"],
            "messages": [
                {
                    "role": "system",
                    "content": "You are a biomedical expert assistant. Provide detailed, accurate answers based on the provided context."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "temperature": 0.2,
            "max_tokens": 800,
            "top_p": 0.95,
            "stream": False
        }

        try:
            response = requests.post(
                config["url"],
                headers=config["headers"],
                json=payload,
                timeout=30
            )

            if response.status_code != 200:
                print(f"⚠️ Error Status: {response.status_code}")
                try:
                    error_json = response.json()
                    error_msg = error_json.get('error', {}).get('message', '')
                    print(f"⚠️ Error Details: {error_msg}")

                    if 'decommissioned' in error_msg.lower() or 'not found' in error_msg.lower():
                        print(f"🔄 Model '{config['model']}' unavailable. Trying fallback...")
                        fallback_models = ["llama-3.1-8b-instant", "gemma2-9b-it", "mixtral-8x7b-32768"]

                        for fallback in fallback_models:
                            if fallback != config["model"]:
                                print(f"   Trying {fallback}...")
                                config["model"] = fallback
                                payload["model"] = fallback

                                response = requests.post(
                                    config["url"],
                                    headers=config["headers"],
                                    json=payload,
                                    timeout=30
                                )

                                if response.status_code == 200:
                                    result = response.json()
                                    print(f"   ✅ Success with {fallback}!")
                                    return result["choices"][0]["message"]["content"].strip()

                        return "Error: All fallback models failed. Please check Groq API status."

                    return f"Groq API Error: {error_msg}"
                except:
                    return f"Groq API Error: Status {response.status_code}"

            response.raise_for_status()
            result = response.json()
            return result["choices"][0]["message"]["content"].strip()

        except Exception as e:
            return f"Error querying Groq: {str(e)}"

    def _query_huggingface(self, prompt: str) -> str:
        """Query HuggingFace API."""
        config = self.configs["huggingface"]

        payload = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 600,
                "temperature": 0.2,
                "top_p": 0.95,
                "return_full_text": False
            }
        }

        try:
            response = requests.post(
                config["url"],
                headers=config["headers"],
                json=payload,
                timeout=30
            )
            response.raise_for_status()
            result = response.json()

            if isinstance(result, list) and len(result) > 0:
                return result[0].get("generated_text", "").strip()
            return result.get("generated_text", "").strip()
        except Exception as e:
            return f"Error querying HuggingFace: {str(e)}"

    def _query_ollama(self, prompt: str) -> str:
        """Query local Ollama instance."""
        config = self.configs["ollama"]

        payload = {
            "model": config["model"],
            "prompt": prompt,
            "stream": False,
            "options": {"temperature": 0.2, "num_predict": 700}
        }

        try:
            response = requests.post(config["url"], json=payload, timeout=60)
            response.raise_for_status()
            return response.json().get("response", "").strip()
        except Exception as e:
            return f"Error querying Ollama: {str(e)}\nMake sure Ollama is running."

    def answer(self, question: str, neo4j_context: str) -> Dict[str, Any]:
        """Generate answer using RAG pipeline."""
        prompt = self._build_prompt(question, neo4j_context)

        if self.backend == "groq":
            answer = self._query_groq(prompt)
        elif self.backend == "huggingface":
            answer = self._query_huggingface(prompt)
        elif self.backend == "ollama":
            answer = self._query_ollama(prompt)
        else:
            answer = f"Unknown backend: {self.backend}"

        return {
            "question": question,
            "answer": answer,
            "context": neo4j_context,
            "backend": self.backend,
            "model": self.configs[self.backend].get("model", "N/A"),
            "prompt_length": len(prompt),
            "context_length": len(neo4j_context)
        }

print("✓ Optimized BiomedicalRAG class defined")

# =============================================================================
# CELL 5: ENHANCED RAG PIPELINE CLASS
# =============================================================================

class CompleteRAGPipeline:
    """End-to-end RAG with smart context retrieval."""

    def __init__(self, neo4j_connector: Neo4jConnector, rag_system: BiomedicalRAG):
        self.neo4j = neo4j_connector
        self.rag = rag_system
        print("✓ Complete RAG pipeline initialized")

    def ask(self, question: str, entity: Optional[str] = None,
            custom_cypher: Optional[str] = None, cypher_params: Optional[Dict] = None,
            use_multihop: bool = False, context_limit: int = 15) -> Dict[str, Any]:
        """Answer question using optimized RAG pipeline."""
        print(f"\n🔍 Processing: {question}")

        if custom_cypher:
            print("📊 Executing custom Cypher query...")
            context = self.neo4j.custom_query_to_context(custom_cypher, cypher_params)
        elif entity:
            print(f"📊 Searching knowledge graph for: {entity}")
            search_mode = "multi-hop" if use_multihop else "enhanced"
            print(f"   Search mode: {search_mode}")
            context = self.neo4j.search_entity(entity, limit=context_limit, use_multihop=use_multihop)
        else:
            words = question.split()
            potential_entities = [w.strip('?,.:;!') for w in words if len(w) > 4 and w[0].isupper()]

            if not potential_entities:
                potential_entities = [w.strip('?,.:;!') for w in words if len(w) > 5]

            if potential_entities:
                entity = potential_entities[0]
                print(f"📊 Auto-extracted entity: '{entity}'")
                context = self.neo4j.search_entity(entity, limit=context_limit, use_multihop=use_multihop)
            else:
                context = "No specific entity detected. Please specify an entity or provide a custom query."

        print(f"✓ Retrieved context ({len(context)} chars, ~{len(context.split())} words)")

        print("🤖 Generating answer...")
        result = self.rag.answer(question, context)
        print("✓ Answer generated\n")

        return result

    def ask_with_fallback(self, question: str, entity: Optional[str] = None) -> Dict[str, Any]:
        """Try enhanced search first, fallback to multi-hop if context insufficient."""
        result = self.ask(question, entity=entity, use_multihop=False, context_limit=15)

        if "not enough information" in result['answer'].lower() or "context doesn't contain" in result['answer'].lower():
            print("⚠️ Initial context insufficient, trying multi-hop search...")
            result = self.ask(question, entity=entity, use_multihop=True, context_limit=20)

        return result

    def close(self):
        self.neo4j.close()

print("✓ Enhanced CompleteRAGPipeline class defined")

# =============================================================================
# CELL 6: CONNECT TO NEO4J
# =============================================================================

print("\n" + "="*70)
print("CONNECTING TO NEO4J AURA")
print("="*70)

NEO4J_URI = "HIDDEN"
NEO4J_USERNAME = "HIDDEN"
NEO4J_PASSWORD = "HIDDEN"
NEO4J_DATABASE = "HIDDEN"

neo4j = Neo4jConnector(
    uri=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE
)

# =============================================================================
# CELL 7: EXPLORE DATABASE
# =============================================================================

print("\n" + "="*70)
print("DATABASE EXPLORATION")
print("="*70)

print("\n📋 Database Schema:")
schema = neo4j.get_schema()
print(schema)

print("\n📊 Sample Data:")
sample = neo4j.get_sample_data(limit=2)
print(sample)

# =============================================================================
# CELL 8: INITIALIZE OPTIMIZED RAG SYSTEM
# =============================================================================

print("\n" + "="*70)
print("INITIALIZING OPTIMIZED RAG SYSTEM")
print("="*70)

GROQ_KEY = "HIDDEN"

rag = BiomedicalRAG(
    backend="groq",
    api_key=GROQ_KEY,
    model="llama-3.1-8b-instant"
)

print("\n🧪 Testing Groq API connection...")
test_result = rag._query_groq("Say 'Hello' in one word.")
if "Error" not in test_result:
    print(f"✅ API working! Response: {test_result}")
else:
    print(f"❌ API test failed: {test_result}")

pipeline = CompleteRAGPipeline(neo4j, rag)

# =============================================================================
# CELL 9: RUN OPTIMIZED EXAMPLE QUERIES
# =============================================================================

print("\n" + "="*70)
print("EXAMPLE QUERIES WITH ENHANCED SEARCH")
print("="*70)

print("\n--- Example 1: Disease Mechanism (Enhanced Search) ---")
result1 = pipeline.ask(
    question="What causes jaundice in cirrhosis?",
    entity="cirrhosis",
    context_limit=20
)
print(f"📝 Question: {result1['question']}")
print(f"🤖 Model: {result1['model']}")
print(f"📊 Context: {result1['context_length']} chars")
print(f"💡 Answer:\n{result1['answer']}\n")

print("\n--- Example 2: Drug Mechanism (Multi-hop Search) ---")
result2 = pipeline.ask(
    question="How does metformin work?",
    entity="metformin",
    use_multihop=True,
    context_limit=20
)
print(f"📝 Question: {result2['question']}")
print(f"💡 Answer:\n{result2['answer']}\n")

print("\n--- Example 3: Disease Symptoms (Fallback Strategy) ---")
result3 = pipeline.ask_with_fallback(
    question="What are common diabetes symptoms?",
    entity="diabetes"
)
print(f"📝 Question: {result3['question']}")
print(f"💡 Answer:\n{result3['answer']}\n")

# =============================================================================
# CELL 10: INTERACTIVE MODE
# =============================================================================

print("\n" + "="*70)
print("INTERACTIVE MODE - READY TO USE")
print("="*70)
print("\n✨ Uncomment the code below for interactive Q&A:")
print("""
while True:
    q = input("\\n\\nYour question (or 'quit'): ")
    if q.lower() in ['quit', 'exit', 'q']:
        break

    entity = input("Entity to search (or press Enter to auto-detect): ").strip() or None
    mode = input("Use multi-hop? (y/n, default=n): ").strip().lower()
    use_multihop = mode == 'y'

    result = pipeline.ask(q, entity=entity, use_multihop=use_multihop, context_limit=20)

    print(f"\\n💡 Answer:\\n{result['answer']}")
    print(f"\\n📊 Context used: {result['context_length']} chars")
""")

# =============================================================================
# CELL 11: ADVANCED USAGE EXAMPLES
# =============================================================================

print("\n" + "="*70)
print("ADVANCED USAGE")
print("="*70)
print("""
# Custom Cypher query for specific relationships
result = pipeline.ask(
    question="What diseases are related to liver dysfunction?",
    custom_cypher=\"\"\"
    MATCH (d:Disease)-[r]->(s)
    WHERE toLower(d.name) CONTAINS 'liver'
    RETURN d.name as disease, type(r) as relationship, s.name as related
    LIMIT 15
    \"\"\"
)

# Multi-hop search for complex queries
result = pipeline.ask(
    question="What are the downstream effects of insulin resistance?",
    entity="insulin",
    use_multihop=True,
    context_limit=25
)

# Fallback strategy for uncertain queries
result = pipeline.ask_with_fallback(
    question="What causes hypertension?",
    entity="hypertension"
)
""")

# =============================================================================
# CELL 12: SYSTEM STATUS & CLEANUP
# =============================================================================

print("\n" + "="*70)
print("SYSTEM STATUS")
print("="*70)
print("✅ All systems operational!")
print("\n🚀 Key Features:")
print("   ✓ Enhanced relevance-based search")
print("   ✓ Multi-hop graph traversal")
print("   ✓ Automatic fallback mechanisms")
print("   ✓ Improved context extraction (15-20 nodes)")
print("   ✓ Better prompting for detailed answers")
print("\n📌 When done: pipeline.close()")
print("\n📚 Available models:")
print("   - llama-3.1-8b-instant (fast, current)")
print("   - gemma2-9b-it (alternative)")
print("   - mixtral-8x7b-32768 (if available)")
print("\n💡 Tips:")
print("   - Use context_limit=20-25 for complex queries")
print("   - Enable use_multihop=True for relationship-heavy questions")
print("   - Try ask_with_fallback() for automatic retry with better context")

✓ All libraries imported successfully
✓ Enhanced Neo4jConnector class defined
✓ Optimized BiomedicalRAG class defined
✓ Enhanced CompleteRAGPipeline class defined

CONNECTING TO NEO4J AURA
✓ Connected to Neo4j at neo4j+s://62418b31.databases.neo4j.io

DATABASE EXPLORATION

📋 Database Schema:
{
  "node_labels": [
    "[Info Tree Node]",
    "[Info Tree Root]",
    "[Disease]",
    "[Embedding Root]",
    "[Embedding Node, pseudo medical history Embedding Node]",
    "[Embedding Node, diagnosis info Embedding Node]",
    "Disease",
    "Info Tree Node",
    "EntityCanonical"
  ],
  "relationship_types": [
    "tree_node-tree_node",
    "tree_root-tree_node",
    "main_node-tree_root",
    "main_node-embedding_root",
    "embedding_root-embedding_node",
    "CANONICAL_FORM",
    "SIMILAR_TO"
  ]
}

📊 Sample Data:
[
  {
    "labels": [
      "[Info Tree Node]"
    ],
    "props": {
      "text": "Diaphragmatic hernia",
      "original_id": "4:77316bc7-08a7-42a4-afdd-da77a7c4cd66:4",
      